In [21]:
!pip install optuna

In [22]:
VERSION = 20

In [23]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

from sklearn.model_selection import train_test_split

from sklearn.neighbors import KNeighborsRegressor
from sklearn.linear_model import LinearRegression
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestRegressor
from sklearn.svm import SVR
from sklearn.metrics import r2_score

from catboost import CatBoostClassifier
from matplotlib import pyplot as plt
from sklearn.metrics import f1_score, roc_auc_score, accuracy_score, precision_score, recall_score
from sklearn.model_selection import train_test_split

In [24]:
df = pd.read_csv(f'train_AIC_processed_v{VERSION}.csv')
df_test = pd.read_csv(f'test_AIC_processed_v{VERSION}.csv')

In [25]:
X_train, y_train = df.drop(columns='y'), df['y']

In [26]:
import optuna
import catboost
from optuna.samplers import TPESampler
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, roc_auc_score, \
    classification_report
def objective(trial):
    model = catboost.CatBoostClassifier(
        iterations=trial.suggest_int("iterations", 100, 25000),
        learning_rate=trial.suggest_float("learning_rate", 1e-3, 1e-1, log=True),
        depth=trial.suggest_int("depth", 4, 10),
        l2_leaf_reg=trial.suggest_float("l2_leaf_reg", 1e-8, 100.0, log=True),
        bootstrap_type=trial.suggest_categorical("bootstrap_type", ["Bayesian"]),
        random_strength=trial.suggest_float("random_strength", 1e-8, 10.0, log=True),
        bagging_temperature=trial.suggest_float("bagging_temperature", 0.0, 10.0),
        od_type=trial.suggest_categorical("od_type", ["IncToDec", "Iter"]),
        od_wait=trial.suggest_int("od_wait", 10, 300),
        verbose=False
    )
    X = df.drop(["y"], axis=1)
    y = df["y"]
    X_train, y_train = X, y
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.01, random_state=1)
    model.fit(X_train, y_train)
    y_pred = model.predict(X_test)
    return accuracy_score(y_test, y_pred)
optuna.logging.set_verbosity(optuna.logging.WARNING)

sampler = TPESampler(seed=1)
study = optuna.create_study(study_name="catboost", direction="maximize", sampler=sampler)
study.optimize(objective, n_trials=1)

print("Number of finished trials: ", len(study.trials))
print("Best trial:")
trial = study.best_trial
print("  Value: ", trial.value)
print("  Params: ")
for key, value in trial.params.items():
    print("    {}: {}".format(key, value))


Number of finished trials:  1
Best trial:
  Value:  0.919196637085474
  Params: 
    iterations: 10484
    learning_rate: 0.027583475549166746
    depth: 4
    l2_leaf_reg: 1.0551779964424746e-05
    bootstrap_type: Bayesian
    random_strength: 2.0931628460945333e-07
    bagging_temperature: 0.923385947687978
    od_type: Iter
    od_wait: 125


In [27]:
ct_model = catboost.CatBoostClassifier(**trial.params, verbose=False)
ct_model.fit(X_train, y_train)
X = df.drop(["y"], axis=1)
y = df["y"]
y_pred = ct_model.predict(X_test)
print(classification_report(y_test, y_pred))
print(accuracy_score(y_test, y_pred))

NameError: name 'X_test' is not defined